In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:

# read in datasets

alq = pd.read_csv('ALQ_L.csv')
bmx = pd.read_csv('BMX_L.csv')
bpxo = pd.read_csv('BPXO_L.csv')
diq = pd.read_csv('DIQ_L.csv')
dpq = pd.read_csv('DPQ_L.csv')
dr1tot = pd.read_csv('DR1TOT_L.csv')
ghb = pd.read_csv('GHB_L.csv')
glu = pd.read_csv('GLU_L.csv')
paq = pd.read_csv('PAQ_L.csv')
smq = pd.read_csv('SMQ_L.csv')
vid = pd.read_csv('VID_L.csv')
demo = pd.read_csv('demo_l.csv')

#upload and fix sas file to be readable
file_path = 'SLQ_L.xpt'
slq = pd.read_sas(file_path, format='xport')
slq = slq.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# function to re
def clean_dataset(df):
    """
    Removes columns with more than 30% missing values and prints missing data stats before and after.

    Parameters:
        df (pd.DataFrame): The dataset to be cleaned.

    Returns:
        pd.DataFrame: The cleaned dataset.
    """
    # Calculate percentage of missing values for pritning if needed
    percent_missing = df.isnull().sum() * 100 / len(df)
    #print(perccent_missing)
    # Define threshold (columns with at least 70% non-missing values are retained)
    threshold = len(df) * 0.7
    df_cleaned = df.dropna(thresh=threshold, axis=1)  # Drop columns below threshold

    # Recalculate missing values for pritning if needed
    percent_missing_after = df_cleaned.isnull().sum() * 100 / len(df_cleaned)
    #print(percent_missing_after)
    return df_cleaned

In [4]:

# function for identifying categorical variables

def convert_cat(df):
    """
    Looks through each variable in df to find its unique values.
    If the number of unique values is less than 9 it converts that variable form a float to an object
    Parameters:
        df (pd.DataFrame): The dataset to be fixed.

    Returns:
        pd.DataFrame: categorical variables converted to object
    """
    
    df = df.copy()  # Avoid modifying the original DataFrame
    
    for col in df.columns:
        if df[col].nunique() < 9:  # Threshold for categorical variables
            if df[col].dtype == 'float64':  # Ensure only float columns are converted
                df[col] = df[col].astype('object')# Convert to categorical type
                df[col] = df[col].round()
    return df


from sklearn.impute import SimpleImputer


def impute(df):

    """
    Looks through each variable in df to find its unique values.
    If the number of unique values is less than 9 it converts that variable form a float to an object
    It the splits the df temporarily based on if the variable is and object or a float data type.
    Usess imputer to fill in missing variables

    Parameters:
        df (pd.DataFrame): The dataset to be cleaned.

    Returns:
        pd.DataFrame: filled null observations
   """
    
    df = df.copy()
                
     # Define imputers for numerical and categorical data
    num_imputer = SimpleImputer(strategy='mean')  # For numerical columns (replace NA with mean)
    cat_imputer = SimpleImputer(strategy='most_frequent')  # For categorical columns (replace NA with mode)

    
    # Separate columns into numerical and categorical
    num_cols = df.select_dtypes(include=['float64']).columns  # All float columns as numerical
    cat_cols = df.select_dtypes(include=['object']).columns  # All object columns as categorical

    # Apply the imputers to the respective columns
    df.loc[:, num_cols] = num_imputer.fit_transform(df.loc[:, num_cols])  # Apply numerical imputation
    df.loc[:, cat_cols] = cat_imputer.fit_transform(df.loc[:, cat_cols])  # Apply categorical imputation

    return df

In [5]:
# Selecting specific columns from each dataset that we want to test
a = alq[["SEQN", "ALQ111"]]
b = bmx[['SEQN', 'BMXBMI', 'BMXWAIST']]
c = bpxo[['SEQN', 'BPXOSY1', 'BPXOSY2', 'BPXOSY3', 'BPXODI1', 'BPXODI2']]
d = diq[['SEQN', 'DIQ010']]
e = dr1tot[['SEQN','DR1TKCAL','DR1TPROT','DR1TCARB','DR1TSUGR','DR1TFIBE',
            'DR1TTFAT','DR1TSFAT','DR1TMFAT','DR1TPFAT','DR1TCHOL','DR1TVB12',
            'DR1TVC','DR1TMAGN','DR1TCAFF','DR1TSODI','DR1TALCO','DR1_320Z']]
f = glu[['SEQN', 'LBXGLU']]
g = smq[['SEQN', 'SMQ020']]
h = vid[['SEQN', 'LBXVIDMS']]
i = dpq[['SEQN', 'DPQ040', 'DPQ050', 'DPQ060']]
j = demo[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMDBORN4',
          'DMDYRUSR', 'DMDEDUC2', 'DMDMARTZ', 'RIDEXPRG', 'INDFMPIR']]
k = ghb[['SEQN','LBXGH']]
l = slq[['SEQN','SLD012']]


In [6]:
#merge into one df
data = j.copy()
for df in [a, b, c, d, e, f, g, h, i, k, l]:  # Removed duplicate merges
    data = pd.merge(data, df, on='SEQN', how='inner')


data_main = convert_cat(data)
print(len(data_main), data_main.shape[1])
print(data_main.info())

3562 44
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3562 entries, 0 to 3561
Data columns (total 44 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      3562 non-null   float64
 1   RIAGENDR  3562 non-null   object 
 2   RIDAGEYR  3562 non-null   float64
 3   RIDRETH3  3562 non-null   object 
 4   DMDBORN4  3562 non-null   object 
 5   DMDYRUSR  744 non-null    object 
 6   DMDEDUC2  3425 non-null   object 
 7   DMDMARTZ  3424 non-null   object 
 8   RIDEXPRG  641 non-null    object 
 9   INDFMPIR  3083 non-null   float64
 10  ALQ111    3049 non-null   object 
 11  BMXBMI    3512 non-null   float64
 12  BMXWAIST  3387 non-null   float64
 13  BPXOSY1   3431 non-null   float64
 14  BPXOSY2   3428 non-null   float64
 15  BPXOSY3   3417 non-null   float64
 16  BPXODI1   3431 non-null   float64
 17  BPXODI2   3428 non-null   float64
 18  DIQ010    3562 non-null   object 
 19  DR1TKCAL  2821 non-null   float64
 20  DR1TPROT  2821 non-nul

In [7]:
unique_counts = data_main.nunique()

print(unique_counts)

SEQN        3562
RIAGENDR       2
RIDAGEYR      63
RIDRETH3       6
DMDBORN4       2
DMDYRUSR       8
DMDEDUC2       6
DMDMARTZ       5
RIDEXPRG       3
INDFMPIR     415
ALQ111         3
BMXBMI       380
BMXWAIST     749
BPXOSY1      122
BPXOSY2      122
BPXOSY3      122
BPXODI1       82
BPXODI2       81
DIQ010         4
DR1TKCAL    1830
DR1TPROT    2656
DR1TCARB    2743
DR1TSUGR    2697
DR1TFIBE     738
DR1TTFAT    2672
DR1TSFAT    2771
DR1TMFAT    2767
DR1TPFAT    2746
DR1TCHOL     817
DR1TVB12    1265
DR1TVC      1759
DR1TMAGN     606
DR1TCAFF     510
DR1TSODI    2168
DR1TALCO     233
DR1_320Z     419
LBXGLU       198
SMQ020         4
LBXVIDMS     887
DPQ040         5
DPQ050         5
DPQ060         5
LBXGH         89
SLD012        23
dtype: int64


In [8]:
data_main.isnull().sum()

SEQN           0
RIAGENDR       0
RIDAGEYR       0
RIDRETH3       0
DMDBORN4       0
DMDYRUSR    2818
DMDEDUC2     137
DMDMARTZ     138
RIDEXPRG    2921
INDFMPIR     479
ALQ111       513
BMXBMI        50
BMXWAIST     175
BPXOSY1      131
BPXOSY2      134
BPXOSY3      145
BPXODI1      131
BPXODI2      134
DIQ010         0
DR1TKCAL     741
DR1TPROT     741
DR1TCARB     741
DR1TSUGR     741
DR1TFIBE     741
DR1TTFAT     741
DR1TSFAT     741
DR1TMFAT     741
DR1TPFAT     741
DR1TCHOL     741
DR1TVB12     741
DR1TVC       741
DR1TMAGN     741
DR1TCAFF     741
DR1TSODI     741
DR1TALCO     741
DR1_320Z     741
LBXGLU       233
SMQ020         1
LBXVIDMS     282
DPQ040       493
DPQ050       494
DPQ060       495
LBXGH        179
SLD012        31
dtype: int64

In [33]:
# Clean dataset
print(len(data_main), data.shape[1])
data_cleaned = clean_dataset(data_main) # Clean dataset
data_cleaned.to_csv("data_cleaned.csv", index=False)
print(len(data_cleaned), data_cleaned.shape[1])
data_imp = impute(data_cleaned)    # fill NAN values
print(len(data_imp), data_imp.shape[1])


3562 44
3562 42
3562 42


In [10]:
for col in data_imp.columns:
    print(f"Value counts for {col}:\n{data_imp[col].value_counts()}\n")

Value counts for SEQN:
SEQN
130378.0    1
138372.0    1
138320.0    1
138323.0    1
138324.0    1
           ..
134446.0    1
134452.0    1
134457.0    1
134461.0    1
142309.0    1
Name: count, Length: 3562, dtype: int64

Value counts for RIAGENDR:
RIAGENDR
2.0    1959
1.0    1603
Name: count, dtype: int64

Value counts for RIDAGEYR:
RIDAGEYR
80.0    204
62.0    106
68.0     95
67.0     88
64.0     88
       ... 
45.0     31
24.0     30
23.0     29
20.0     29
78.0     27
Name: count, Length: 63, dtype: int64

Value counts for RIDRETH3:
RIDRETH3
3.0    2086
4.0     424
2.0     365
1.0     265
7.0     220
6.0     202
Name: count, dtype: int64

Value counts for DMDBORN4:
DMDBORN4
1.0    2818
2.0     744
Name: count, dtype: int64

Value counts for DMDEDUC2:
DMDEDUC2
5.0    1378
4.0    1029
3.0     730
2.0     256
1.0     167
9.0       2
Name: count, dtype: int64

Value counts for DMDMARTZ:
DMDMARTZ
1.0     2052
2.0      843
3.0      664
99.0       2
77.0       1
Name: count, dtype: int64

In [11]:
print(data_imp.info())  #check non null count and Dtype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3562 entries, 0 to 3561
Data columns (total 42 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      3562 non-null   float64
 1   RIAGENDR  3562 non-null   object 
 2   RIDAGEYR  3562 non-null   float64
 3   RIDRETH3  3562 non-null   object 
 4   DMDBORN4  3562 non-null   object 
 5   DMDEDUC2  3562 non-null   object 
 6   DMDMARTZ  3562 non-null   object 
 7   INDFMPIR  3562 non-null   float64
 8   ALQ111    3562 non-null   object 
 9   BMXBMI    3562 non-null   float64
 10  BMXWAIST  3562 non-null   float64
 11  BPXOSY1   3562 non-null   float64
 12  BPXOSY2   3562 non-null   float64
 13  BPXOSY3   3562 non-null   float64
 14  BPXODI1   3562 non-null   float64
 15  BPXODI2   3562 non-null   float64
 16  DIQ010    3562 non-null   object 
 17  DR1TKCAL  3562 non-null   float64
 18  DR1TPROT  3562 non-null   float64
 19  DR1TCARB  3562 non-null   float64
 20  DR1TSUGR  3562 non-null   floa

In [12]:
adj_data = data_imp.copy() #prep new data for outlier removal and adding variables for usability

In [13]:
adj_data.drop(columns=['SEQN'], inplace=True)    # remove ID variable SEQN, only needed for join

#convert 'I dont Know' values to no
adj_data['DMDMARTZ'] = adj_data['DMDMARTZ'].replace(99, 3)   
adj_data['ALQ111'] = adj_data['ALQ111'].replace(9, 2)
adj_data['SMQ020'] = adj_data['SMQ020'].replace(9, 2)
adj_data['DPQ050'] = adj_data['DPQ050'].replace(9, 0)

adj_data['DIQ010'] = adj_data['DIQ010'].replace(3, 1)     # Diagnosis variable, convert borderline( prediabetic diagnosis) value
                                                          #  to Diabetic value for data clarity.
# remove outlires, between 1 and 5 observations per variable listed below
adj_data = adj_data[adj_data['DMDEDUC2'] != 9]
adj_data = adj_data[adj_data['DMDMARTZ'] != 77]
adj_data = adj_data[adj_data['DIQ010'] != 9]
adj_data = adj_data[adj_data['SMQ020'] != 7]
adj_data = adj_data[adj_data['DPQ040'] != 7]
adj_data = adj_data[adj_data['DPQ060'] != 7]

print(len(adj_data), adj_data.shape[1])   # check how many observations were lost, only 11

adj_data = convert_cat(adj_data)     #convert  category values again in case any reverted back to float Dtype

print(adj_data.info())   

3551 41
<class 'pandas.core.frame.DataFrame'>
Index: 3551 entries, 0 to 3561
Data columns (total 41 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   RIAGENDR  3551 non-null   object 
 1   RIDAGEYR  3551 non-null   float64
 2   RIDRETH3  3551 non-null   object 
 3   DMDBORN4  3551 non-null   object 
 4   DMDEDUC2  3551 non-null   object 
 5   DMDMARTZ  3551 non-null   object 
 6   INDFMPIR  3551 non-null   float64
 7   ALQ111    3551 non-null   object 
 8   BMXBMI    3551 non-null   float64
 9   BMXWAIST  3551 non-null   float64
 10  BPXOSY1   3551 non-null   float64
 11  BPXOSY2   3551 non-null   float64
 12  BPXOSY3   3551 non-null   float64
 13  BPXODI1   3551 non-null   float64
 14  BPXODI2   3551 non-null   float64
 15  DIQ010    3551 non-null   object 
 16  DR1TKCAL  3551 non-null   float64
 17  DR1TPROT  3551 non-null   float64
 18  DR1TCARB  3551 non-null   float64
 19  DR1TSUGR  3551 non-null   float64
 20  DR1TFIBE  3551 non-null   f

In [14]:

adj_data.to_csv("adj_data.csv", index=False)